# Operando con datos en Pandas

Una de las piezas esenciales de NumPy es la capacidad de realizar operaciones rápidas entre elementos, tanto con aritmética básica (suma, resta, multiplicación, etc.) como con operaciones más sofisticadas (funciones trigonométricas, funciones exponenciales y logarítmicas, etc.).
**Pandas hereda gran parte de esta funcionalidad de NumPy y las ufuncs** que presentamos en [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) son claves para esto.

Sin embargo, Pandas incluye un par de cambios útiles: para operaciones unarias como negación y funciones trigonométricas, estas ufuncs *preservarán las etiquetas de índice y columna* en la salida, y para operaciones binarias como **suma y multiplicación, Pandas *alineará automáticamente los índices *al pasar los objetos al ufunc.**
Esto significa que mantener el contexto de los datos y combinar datos de diferentes fuentes (ambas tareas potencialmente propensas a errores con matrices NumPy sin procesar) se vuelven esencialmente infalibles con Pandas.
Veremos adicionalmente que existen operaciones bien definidas entre estructuras unidimensionales ``Series`` y estructuras bidimensionales ``DataFrame``.

## Ufuncs: Preservación del índice

Debido a que Pandas está diseñado para funcionar con NumPy, cualquier ufunc de NumPy funcionará en los objetos ``Series`` y ``DataFrame`` de Pandas.
Comencemos definiendo una ``Serie`` y un ``DataFrame`` simples para demostrar esto:

In [5]:
import pandas as pd
import numpy as np

In [6]:
rng = np.random.RandomState(42) #Esto es como una semilla. Como el seed en numpy.
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int32

In [7]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


Si aplicamos una ufunc NumPy en cualquiera de estos objetos, el resultado será otro objeto Pandas *con los índices conservados:*

In [8]:
ser

0    6
1    3
2    7
3    4
dtype: int32

In [9]:
np.exp(ser) #calcula la exponencial de todos los valores

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

O, para un cálculo un poco más complejo:

In [10]:
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


In [11]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


Cualquiera de las funciones discutidas en [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) se puede utilizar de manera similar.

## UFuncs: Alineación del índice

Para operaciones binarias en dos objetos ``Series`` o ``DataFrame``, Pandas alineará los índices en el proceso de realizar la operación.
Esto es muy conveniente cuando se trabaja con datos incompletos, como veremos en algunos de los ejemplos siguientes.

### Alineación de índice en serie

Como ejemplo, supongamos que combinamos dos fuentes de datos diferentes y encontramos solo los tres principales estados de EE. UU. por *área* y los tres principales estados de EE. UU. por *población*:

In [12]:
area = pd.Series({'Alaska': 1723337,
                  'Texas': 695662,
                  'California': 423967},
                 name='area')

population = pd.Series({'California': 38332521,
                        'Texas': 26448193,
                        'New York': 19651127},
                       name='population')

Veamos qué sucede cuando los dividimos para calcular la densidad de población:

In [13]:
type(population / area)

pandas.core.series.Series

In [14]:
s_poblacion_area = population / area
print(s_poblacion_area)

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64


La matriz resultante contiene la *unión* de índices de las dos matrices de entrada, que podrían determinarse usando la aritmética de conjuntos estándar de Python en estos índices:

In [15]:
area.index

Index(['Alaska', 'Texas', 'California'], dtype='object')

In [16]:
population.index

Index(['California', 'Texas', 'New York'], dtype='object')

In [17]:
area.index | population.index #Si queremos hacer esta aplicacion debemos pasarlo a set

TypeError: unsupported operand type(s) for |: 'str' and 'str'

In [ ]:
set(area.index) | set(population.index)

{'Alaska', 'California', 'New York', 'Texas'}

In [ ]:
area.index.union(population.index)

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

Cualquier elemento para el cual uno u otro no tenga una entrada se marca con ``NaN``, o "No es un número", que es como Pandas marca los datos faltantes (consulte más información sobre los datos faltantes en [Handling Missing Data](03.04-Missing-Values.ipynb)).
Esta coincidencia de índice se implementa de esta manera para cualquiera de las expresiones aritméticas integradas de Python; cualquier valor faltante se completa con NaN de forma predeterminada:

In [ ]:
A = pd.Series([2, 4, 6], index=["andalucia", "aragon", "madrid"])
print(A)
B = pd.Series([1, 3, 5], index=["aragon", "madrid", "asturias"])
print(B)
A + B

andalucia    2
aragon       4
madrid       6
dtype: int64
aragon      1
madrid      3
asturias    5
dtype: int64


andalucia    NaN
aragon       5.0
asturias     NaN
madrid       9.0
dtype: float64

In [ ]:
df_AB = pd.DataFrame({'n_alumnos': A, 'n_profesores': B})
df_AB

,n_alumnos,n_profesores
andalucia,2.0,NaN
aragon,4.0,1.0
asturias,NaN,5.0
madrid,6.0,3.0


In [ ]:
# df_AB = pd.DataFrame({'n_alumnos': A, 'n_profesores': B})
df_AB['n_personas'] = df_AB['n_alumnos'] + df_AB['n_profesores']
df_AB

,n_alumnos,n_profesores,n_personas
andalucia,2.0,NaN,NaN
aragon,4.0,1.0,5.0
asturias,NaN,5.0,NaN
madrid,6.0,3.0,9.0


In [ ]:
df_AB['ratio_a'] = df_AB['n_alumnos'] / df_AB['n_profesores']
df_AB

,n_alumnos,n_profesores,n_personas,ratio_a
andalucia,2.0,NaN,NaN,NaN
aragon,4.0,1.0,5.0,4.0
asturias,NaN,5.0,NaN,NaN
madrid,6.0,3.0,9.0,2.0


In [ ]:
df_AB['n_personas'].fillna(0) #fillna sustituye los valores ausentes NaN con el dato que queremos 

andalucia    0.0
aragon       5.0
asturias     0.0
madrid       9.0
Name: n_personas, dtype: float64

Si usar valores NaN no es el comportamiento deseado, el valor de relleno se puede modificar usando métodos de objeto apropiados en lugar de los operadores.
Por ejemplo, llamar a ``A.add(B)`` es equivalente a llamar a ``A + B``, pero permite la especificación explícita opcional del valor de relleno para cualquier elemento en ``A`` o ``B`` que podría faltar:

In [ ]:
B + A

andalucia    NaN
aragon       5.0
asturias     NaN
madrid       9.0
dtype: float64

In [ ]:
print(A)
print(B)

andalucia    2
aragon       4
madrid       6
dtype: int64
aragon      1
madrid      3
asturias    5
dtype: int64


In [ ]:
serie_1 = A.add(B, fill_value=0) #el add sustituye el NAN con el valor indicado en este caso 0. fill_value=0
print(serie_1)

andalucia    2.0
aragon       5.0
asturias     5.0
madrid       9.0
dtype: float64


### Alineación de índice en DataFrame

Un tipo similar de alineación tiene lugar para *ambas* columnas e índices cuando se realizan operaciones en ``DataFrame``s:

In [ ]:
df_A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB')) #Por defecto las operaciones se hacen a nivel de registrp
df_A

,A,B
0,19,2
1,4,18


In [ ]:
list('BAC')

['B', 'A', 'C']

In [ ]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

,B,A,C
0,4,0,9
1,5,8,0
2,9,2,6


In [ ]:
df_A

,A,B
0,19,2
1,4,18


In [ ]:
df_A + B

,A,B,C
0,19.0,6.0,NaN
1,12.0,23.0,NaN
2,NaN,NaN,NaN


In [ ]:
A.add(B, fill_value=0)

,A,B,C
0,1.0,15.0,9.0
1,13.0,6.0,0.0
2,2.0,9.0,6.0


Observe que los índices están alineados correctamente independientemente de su orden en los dos objetos y los índices del resultado están ordenados.
Como fue el caso con ``Series``, podemos usar el método aritmético del objeto asociado y pasar cualquier ``fill_value`` que deseemos para usar en lugar de las entradas faltantes.
Aquí completaremos con la media de todos los valores en ``A`` (calculada apilando primero las filas de ``A``):

In [ ]:
df_A.values.mean() #Media de TODOS los valores

10.75

In [ ]:
df_A

,A,B
0,19,2
1,4,18


In [ ]:
B

,B,A,C
0,4,0,9
1,5,8,0
2,9,2,6


In [ ]:
fill = df_A.values.mean()
df_A.add(B, fill_value=fill) #En aquellos huecos que no tenemos valores lo llenamos con la media

,A,B,C
0,19.00,6.00,19.75
1,12.00,23.00,10.75
2,12.75,19.75,16.75


La siguiente tabla enumera los operadores de Python y sus métodos de objeto Pandas equivalentes:

| Python Operator | Pandas Method(s)                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## Ufuncs: Operaciones entre DataFrame y Series

Al realizar operaciones entre un ``DataFrame`` y una ``Series``, la alineación del índice y la columna se mantiene de manera similar.
Las operaciones entre un ``DataFrame`` y una ``Series`` son similares a las operaciones entre una matriz NumPy bidimensional y unidimensional.
Considere una operación común, donde encontramos la diferencia entre una matriz bidimensional y una de sus filas:

In [ ]:
A = rng.randint(10, size=(3, 4))
A

array([[7, 4, 1, 4],
       [7, 9, 8, 8],
       [0, 8, 6, 8]])

In [ ]:
A - A[0] #Por defecto lo hace por FILAS

array([[ 0,  0,  0,  0],
       [ 0,  5,  7,  4],
       [-7,  4,  5,  4]])

Según las reglas de transmisión de NumPy (ver [Computation on Arrays: Broadcasting](02.05-Computation-on-arrays-broadcasting.ipynb)), la resta entre una matriz bidimensional y una de sus filas se aplica fila por fila.

En Pandas, la convención opera de manera similar en filas de forma predeterminada:

In [ ]:
df = pd.DataFrame(A, columns=list('QRST'))
print(df)
df - df.iloc[0]

   Q  R  S  T
0  7  4  1  4
1  7  9  8  8
2  0  8  6  8


,Q,R,S,T
0,0,0,0,0
1,0,5,7,4
2,-7,4,5,4


Si, en cambio, desea operar en columnas, puede utilizar los métodos de objeto mencionados anteriormente, mientras especifica la palabra clave ``axis``:

In [ ]:
df

,Q,R,S,T
0,7,4,1,4
1,7,9,8,8
2,0,8,6,8


In [ ]:
df = df.subtract(df['R'], axis=0) #Con el metodo subtract estamos eliminando 'R' a cada una de sus columnas. Axis=0 (nivel columnar, la operacion va por filas)
#axis=0 FILAS: en  este caso aunke borremos la columna R la operacion va por filas!!

In [ ]:
df

,Q,R,S,T
0,3,0,-3,0
1,-2,0,-1,-1
2,-8,0,-2,0


Tenga en cuenta que estas operaciones ``DataFrame``/``Series``, como las operaciones analizadas anteriormente, alinearán automáticamente los índices entre los dos elementos:

In [19]:
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


In [18]:
halfrow = df.iloc[0, ::2]
halfrow

A    6
C    2
Name: 0, dtype: int32

In [ ]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-5.0,NaN,2.0,NaN
2,-11.0,NaN,1.0,NaN


In [ ]:
df.subtract(halfrow) #Lo hace por filas pero el sentido es columnar

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-5.0,NaN,2.0,NaN
2,-11.0,NaN,1.0,NaN


In [ ]:
#loc: hay que indicar las columnas por los nombres (etiquetas)

Esta preservación y alineación de índices y columnas significa que las operaciones con datos en Pandas siempre mantendrán el contexto de los datos, lo que evita los tipos de errores tontos que pueden surgir al trabajar con datos heterogéneos y/o desalineados en matrices NumPy sin procesar.